In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Bidirectional, Dense, Dropout
import matplotlib.pyplot as plt

# Завантаження та попередня обробка даних
tags_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\tags.csv')
movies_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\movies.csv')
links_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\links.csv')
ratings_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\ratings_filtered.csv')

# Призначте унікальний ідентифікатор для кожного фільму
movies_df['movieId'] = pd.factorize(movies_df['title'])[0]

# Об'єднайте файли на основі movieId
merged_df = movies_df.merge(links_df, on='movieId', how='inner')
merged_df = merged_df.merge(tags_df, on='movieId', how='inner')
merged_df = merged_df.merge(ratings_df[['userId', 'movieId', 'rating']], on=['userId', 'movieId'], how='inner')

# Визначте рейтинги як позитивні або негативні за певним порогом
threshold = 3.5
merged_df['positive'] = (merged_df['rating'] >= threshold).astype(int)

# Відберіть необхідні колонки та попередньо обробіть дані
selected_df = merged_df[['positive', 'tag', 'title']]
selected_df['text'] = selected_df['tag'] + ' ' + selected_df['title']
selected_df = selected_df.drop(['tag', 'title'], axis=1)

# Використовуємо згортковий метод для обмеження кількості слів
max_words = 5000
tokenizer = imdb.get_tokenizer(num_words=max_words)
x_train = tokenizer.texts_to_sequences(selected_df['text'])
x_train = pad_sequences(x_train, maxlen=300)
y_train = selected_df['positive'].values

# Побудова моделі з використанням RNN, LSTM, Bidirectional LSTM та глибокого LSTM
models = []

# RNN
model_rnn = Sequential([
    Embedding(max_words, 128, input_length=300),
    SimpleRNN(128),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
models.append(('RNN', model_rnn))

# LSTM
model_lstm = Sequential([
    Embedding(max_words, 128, input_length=300),
    LSTM(128),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
models.append(('LSTM', model_lstm))

# Bidirectional LSTM
model_bidirectional_lstm = Sequential([
    Embedding(max_words, 128, input_length=300),
    Bidirectional(LSTM(128)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
models.append(('Bidirectional LSTM', model_bidirectional_lstm))

# Deep LSTM
model_deep_lstm = Sequential([
    Embedding(max_words, 128, input_length=300),
    LSTM(128, return_sequences=True),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
models.append(('Deep LSTM', model_deep_lstm))

# Навчання моделей та візуалізація результатів
for model_name, model in models:
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=2)

    # Візуалізація точності та функції втрат під час навчання
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f'{model_name} Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'{model_name} Training and Validation Loss')

    plt.show()
